In [388]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [389]:
df = pd.read_pickle(r"C:\Users\dpine\red_electrica\red_electrica\data\df_2014_2025.pickle")

In [390]:
df.set_index('date', inplace=True)

In [391]:
# Agrupar por hora y calcular la media
df_hourly = df.resample('H').mean()

C:\Users\dpine\AppData\Local\Temp\ipykernel_4240\3965767037.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample('H').mean()


In [392]:
df_hourly = df_hourly['value'].interpolate(method='time')

In [393]:
df_hourly = df_hourly.to_frame()

In [394]:
df_hourly.isna().sum()

value    0
dtype: int64

In [395]:
df_hourly.duplicated().sum()

np.int64(28528)

In [396]:
df_hourly.drop_duplicates(inplace=True)

In [397]:
df = df_hourly.copy()

In [398]:
df.reset_index(inplace=True)

# Añadimos variables de interés

In [399]:
df["hora"] = df.date.dt.hour

In [400]:
df["dia_semana"] = df.date.dt.weekday

In [401]:
df["mes"] = df.date.dt.month

In [402]:
df["trimestre"] = df.date.dt.quarter

In [403]:
df["año"] = df.date.dt.year

In [404]:
import holidays

es_holidays = holidays.Spain(years=[2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025], observed = True)

df["festivo"] = df["date"].dt.date.isin(es_holidays.keys()).astype(int)

In [405]:
df['fin_de_semana'] = (df['dia_semana'] >= 5).astype(int)

In [406]:
df["es_navidad_fin_anio"] = (((df["date"].dt.month == 12) & (df["date"].dt.day.isin([31])))).astype(int)

In [407]:
df["es_verano"] = df.mes.isin([6, 7, 8]).astype(int)
df["es_navidad"] = df.mes.isin([12, 1]).astype(int)

In [408]:
pib = pd.read_excel(r"C:\Users\dpine\red_electrica\red_electrica\data\PIB_trimestral_excel.xlsx")
pib = pib.iloc[8:52,0:2]
dato = pib.loc[pib["Series de resultados detallados"] == "2024T4"]["Unnamed: 1"].values[0] * 1.06
dato = pd.DataFrame({'Series de resultados detallados': ["2025T1"], 'Unnamed: 1': [dato]})  # Nueva fila como DataFrame
pib = pd.concat([pib, dato], ignore_index=True)

df['año_trimestre'] = df['año'].astype(str) + 'T' + df['trimestre'].astype(str)
df = df.merge(pib, how= "left", left_on = "año_trimestre", right_on = "Series de resultados detallados")
df.drop(columns = ["año_trimestre", "Series de resultados detallados"], inplace = True)
df.rename(columns = {"Unnamed: 1":"pib_trimestral"}, inplace = True)

c:\Users\dpine\red_electrica\red_electrica\venv2\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Variables de lag

In [409]:
def crear_lags(df, variable, num_lags = 7):
    
    #Crea el objeto dataframe
    lags = pd.DataFrame()
    
    #Crea todos los lags
    for cada in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(cada)] = df[variable].shift(cada)
    
    #Devuelve el dataframe de lags
    return(lags)

In [410]:
lags_120 = crear_lags(df, variable="value", num_lags=48)

In [411]:
#df['lag_168'] = df['value'].shift(24*7)

## Variables de medias móviles

In [412]:
def min_movil(df, variable, num_periodos = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()
    
    #Devuelve el dataframe de lags
    return(minm)

In [413]:
def media_movil(df, variable, num_periodos = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()
    
    #Devuelve el dataframe de lags
    return(mm)

In [414]:
def max_movil(df, variable, num_periodos = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()
    
    #Devuelve el dataframe de lags
    return(maxm)

In [415]:
value_min_movil_df = min_movil(df, variable = 'value', num_periodos=24)

In [416]:
value_mean_movil_df = media_movil(df, variable = 'value', num_periodos= 24)

In [417]:
value_max_movil_df = max_movil(df, variable = 'value', num_periodos= 24)

In [418]:
#value_mean_movil_df_168 = df['value'].shift(1).rolling(168).mean()
#value_max_movil_df_168 = df['value'].shift(1).rolling(168).max()
#value_min_movil_df_168 = df['value'].shift(1).rolling(168).min()

## Unimos todo

In [419]:
df_todo = pd.concat([df, lags_120, value_min_movil_df, value_mean_movil_df, value_max_movil_df], axis = 1)

In [420]:
df_todo.duplicated().sum()

np.int64(0)

In [421]:
df_todo.dropna(inplace=True)

# One Hot Encoding

In [422]:
from sklearn.preprocessing import OneHotEncoder

In [423]:
var_ohe = ['hora',
          'dia_semana',
          'mes',
          'trimestre',
          'año',
          'festivo',
          "fin_de_semana",
          "es_navidad_fin_anio",
          "es_verano",
          "es_navidad",  
        ]

In [424]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

In [425]:
df_todo_ohe = ohe.fit_transform(df_todo[var_ohe])

In [426]:
df_todo_ohe = pd.DataFrame(df_todo_ohe, columns = ohe.get_feature_names_out())

# Juntamos los dataframes

In [431]:
df_definitivo = pd.concat([df_todo, df_todo_ohe], axis = 1)

In [432]:
df_definitivo.dropna(inplace=True)

In [433]:
df_definitivo.duplicated().sum()

np.int64(0)

In [434]:
#df_definitivo.drop(columns = ["hora","dia_semana","mes","trimestre","año","festivo","fin_de_semana", "es_navidad_fin_anio", "es_verano", "es_navidad"], inplace = True)

In [435]:
df_definitivo.to_pickle(r"C:\Users\dpine\red_electrica\red_electrica\data\df_definitivo.pickle")

In [436]:
df_definitivo.to_csv(r"C:\Users\dpine\red_electrica\red_electrica\data\df_definitivo2.csv")

precio electricidad, temperatura cada mes

atipicos, describe()